In [ ]:
# # Add the root directory to the path to allow importing the module
import sys
# sys.path.append('/mn/stornext/u3/avijeetp/codes/ISPy')
sys.path.append('/mn/stornext/u3/avijeetp/codes/pyMilne')

In [ ]:
import numpy as np
import MilneEddington as ME
import crisp
import time
from inv_utils import getWavelengths, loadFits, loadCmap, get_nan_mask
from inv_utils import findgrid, plot_output

In [ ]:

class container:
    def __init__(self):
        pass

In [ ]:
#
# Decide to work in float32 or float64
#
dtype = 'float32'
nthreads = 80

In [ ]:
datadir = '/mn/stornext/d18/lapalma/reduc/2020/2020-08-07/CRISP/cubes_nb/'
crisp_im_file = 'nb_6173_2020-08-07T08:22:14_scans=0-56_stokes_corrected_im.fits'
scale = 0.059 # arcsec/pixel
# datadir = '/mn/stornext/d18/lapalma/reduc/2024/2024-05-21/CRISP/cubes_nb/'
# crisp_im_file = 'nb_6173_2024-05-21T10:19:04_10:19:04=0-52_stokes_corrected_im.fits'
crisp_im = datadir + crisp_im_file

In [ ]:

#
# Load data, wavelength array and cmap
#
l = container()
container.iwav = getWavelengths(crisp_im)
container.d = loadFits(crisp_im, tt=0) 
container.cmap = loadCmap(crisp_im, tt=0)

In [ ]:
mask = get_nan_mask(crisp_im, tt=0)
mask.shape

In [ ]:
# Minimum step:
dw = np.min(np.diff(l.iwav))
# dw = round((lambda*10. - lc) * 1000.) ; offset in mA
dw = round(dw*1000.)/1000. # avoid floating point errors


In [ ]:

# The inversions need to account for the instrumental
# profile, which involve convolutions. The convolutions
# must be done in a wavelength grid that is at least
# 1/2 of the FWHM of the instrumental profile. In the
# case of CRISP that would be ~55 mA / 2 = ~27.5 mA
#
# Get finer grid for convolutions purposes
# Since we only observed at the lines, let's create
# two regions, one for each line
#
# The observed line positions are not equidistant, the
# Fe I 6301 points only fit into a regular grid of 5 mA
# whereas the Fe I 6302 can fit into a 15 mA grid
#
iw, idx = findgrid(l.iwav, dw)  # Fe I 6173


In [ ]:

#
# Now we need to create a data cube with the fine grid
# dimensions. All observed points will contribute to the
# inversion. The non-observed ones will have zero weight
# but will be used internally to properly perform the
# convolution of the synthetic spectra
#


ny, nx = l.d.shape[0:2]
obs = np.zeros((ny, nx, 4, iw.size), dtype=dtype, order='c')

for ss in range(4):
    for ii in range(idx.size):
        obs[:, :, ss, idx[ii]] = l.d[:, :, ss, ii]


In [ ]:

#
# Create sigma array with the estimate of the noise for
# each Stokes parameter at all wavelengths. The extra
# non-observed points will have a very large noise (1.e34)
# (zero weight) compared to the observed ones (3.e-3)
# Since the amplitudes of Stokes Q,U and V are very small
# they have a low imprint in Chi2. We can artificially
# give them more weight by lowering the noise estimate.
#
sig = np.zeros((4, iw.size), dtype=dtype) + 1.e32
sig[:, idx] = 5.e-3
sig[1:2, idx] /= 9.0
sig[3, idx] /= 4.0



In [ ]:

#
# Init Me class. We need to create two regions with the
# wavelength arrays defined above and a instrumental profile
# for each region in with the same wavelength step
#
tw = (np.arange(iw.size, dtype=dtype)-iw.size//2)*dw


In [ ]:
# Central wavelength of the line:
l0 = iw[iw.size//2]
tr = crisp.crisp(l0).dual_fpi(tw, erh=-0.001)

regions = [[iw, tr/tr.sum()]]
lines = [int(l0)]
me = ME.MilneEddington(regions, lines, nthreads=nthreads, precision=dtype)


In [ ]:

#
# Init model parameters
#
iPar = np.float64([1500, 2.2, 1.0, -0.5, 0.035, 50., 0.1, 0.24, 0.7]) # [B_tot, theta_B, chi_B, gamma_B, v_los, eta_0, Doppler width, damping, s0, s1]
Imodel = me.repeat_model(iPar, ny, nx)


In [ ]:
#
# Run a first cycle with 4 inversions of each pixel (1 + 3 randomizations) of simple pixel-wise inversion
#
t0 = time.time()
Imodel, syn, chi2 = me.invert(
    Imodel, obs, sig, nRandom=6, nIter=25, chi2_thres=1.0, mu=0.54184232)
t1 = time.time()
print("dT = {0}s -> <Chi2> = {1}".format(t1-t0, chi2.mean()))

In [ ]:
Imodel.shape

In [ ]:
plot_output(Imodel,mask,scale=scale)

In [ ]:
#
# Run second cycle
#
t0 = time.time()
Imodel, syn, chi2 = me.invert(
    Imodel, obs, sig, nRandom=1, nIter=50, chi2_thres=1.0, mu=0.54184232)
t1 = time.time()
print("dT = {0}s -> <Chi2> = {1}".format(t1-t0, chi2.mean()))

In [ ]:
plot_output(Imodel,mask,scale=scale)

In [ ]:

#
# Run a first cycle with 4 inversions of each pixel (1 + 3 randomizations)
#
t0 = time.time()
mo, syn, chi2 = me.invert_spatially_regularized(Imodel, obs, sig,  nIter=25, chi2_thres=1.0, mu=0.54184232, alpha=30., alphas=np.float32([
                                                2, 0.5, 2, 0.01, 0.1, 0.01, 0.1, 0.01, 0.01]), method=1, delay_bracket=3)
t1 = time.time()
print("dT = {0}s -> <Chi2> (including regularization) = {1}".format(t1-t0, chi2))


In [ ]:

#
# Correct velocities for cavity error map from CRISP
#
mos = np.squeeze(mo) # Remove the singleton dimension in the model and make the shape (ny, nx, 9) from (1, ny, nx, 9)
mos[:,:,3] += (l.cmap * 10) / l0 * 2.9e5
# mos[:,:,3] += l.cmap+0.45 # The 0.45 is a global offset that seems to make the umbra at rest


In [ ]:
plot_output(mos,mask,scale=scale)